In [ ]:
!pip install google-genai datasets pandas tqdm

In [ ]:
from google import genai
from google.genai import types
import time
from datasets import load_dataset
import pandas as pd
from tqdm.notebook import tqdm

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

print("genAI Client iniciado")

genAI Client iniciado


In [ ]:
NUM_TEST_SAMPLES = 100
print("Loading ag_news dataset...")
ag_news_dataset = load_dataset("ag_news")
# Use the official test split for evaluation
test_data = ag_news_dataset['test']

# --- !! Select a subset for faster testing initially !! ---
# Reduce this number for quick tests, increase for full evaluation
test_sample = test_data.shuffle(seed=42).select(range(NUM_TEST_SAMPLES))
print(f"Using {len(test_sample)} samples from the test set.")

# Define labels and categories
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
category_names = list(label_map.values())

Loading ag_news dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Using 100 samples from the test set.


In [ ]:
# --- 3. Prepare Few-Shot Examples (CRITICAL: Use TRAINING data, NOT test data) ---
print("Preparing few-shot examples...")
train_data = ag_news_dataset['train']
# Choose 3 diverse examples from the training set
few_shot_examples_raw = train_data.shuffle(seed=42).select([0, 100, 200]) # Example indices

few_shot_prompt_examples = ""
for example in few_shot_examples_raw:
    text = example['text'].strip()
    label_num = example['label']
    label_name = label_map[label_num]
    # Limit example text length to avoid overly long prompts
    text_snippet = text[:500] + "..." if len(text) > 500 else text
    few_shot_prompt_examples += f"Example Article:\n'''{text_snippet}'''\nCategory: {label_name}\n\n---\n\n"

Preparing few-shot examples...


In [ ]:
# --- 4. Define Prompting Functions ---
def create_zeroshot_prompt(article_text, categories):
    instruction = f"Classify the following news article into one of these categories: {', '.join(categories)}."
    # Use triple quotes for potentially multi-line articles
    return f"{instruction}\n\nArticle:\n'''{article_text}'''\n\nCategory:"

def create_fewshot_prompt(article_text, categories, examples_string):
    instruction = f"Classify the following news article into one of these categories: {', '.join(categories)}. Follow the format of the examples provided."
    return f"{instruction}\n\n{examples_string}Now classify this article:\n\nArticle:\n'''{article_text}'''\n\nCategory:"

In [ ]:
# --- 5. Function to Call Gemini API and Parse Response ---
# Use the latest stable Flash model
# model = genai.GenerativeModel('gemini-1.5-flash-latest')

def get_gemini_classification(prompt):
    start_time = time.time()
    predicted_category = None
    raw_response = "Error: No response"
    latency = 0

    try:
        # Set safety to lowest and temperature to 0 for deterministic classification
        safety_settings = [
            types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_HARASSMENT, threshold=types.HarmBlockThreshold.BLOCK_NONE),
            types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH, threshold=types.HarmBlockThreshold.BLOCK_NONE),
            types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT, threshold=types.HarmBlockThreshold.BLOCK_NONE),
            types.SafetySetting(category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT, threshold=types.HarmBlockThreshold.BLOCK_NONE),
        ]
        generation_config = types.GenerateContentConfig(
            temperature=0.0,
            max_output_tokens=10 # Limit output tokens for classification
            )
        generation_config.safety_settings= safety_settings
        response = client.models.generate_content( # Call generate_content on the model object
            model='gemini-2.0-flash-001',
            contents=prompt,
            config=generation_config,
        )
        end_time = time.time()
        latency = end_time - start_time
        raw_response = response.text.strip()

        # --- Basic Response Cleaning ---
        # Find the first category name mentioned in the response (case-insensitive)
        cleaned_response_lower = raw_response.lower()
        for cat_name in category_names:
            if cat_name.lower() in cleaned_response_lower:
                predicted_category = cat_name
                break # Take the first match

        if predicted_category is None:
             print(f"Warning: Could not parse category from response: '{raw_response}' for prompt: '{prompt[:100]}...'")


    except Exception as e:
        latency = time.time() - start_time # Record time even on error
        raw_response = f"API Error: {e}"
        print(f"Error calling Gemini API: {e}")
        # Consider adding a retry mechanism here for production

    # Add a small delay to avoid hitting rate limits
    time.sleep(4) # Sleep for 1 second between calls

    return predicted_category, latency, raw_response

In [ ]:
# --- 6. Run Evaluation Loop ---
results_zeroshot = []
results_fewshot = []

print("\nStarting evaluation...")
for example in tqdm(test_sample): # Wrap test_sample with tqdm for progress bar
    true_label_num = example['label']
    true_label_name = label_map[true_label_num]
    article_text = example['text'].strip()

    # --- Zero-Shot ---
    prompt_zs = create_zeroshot_prompt(article_text, category_names)
    pred_zs, lat_zs, raw_zs = get_gemini_classification(prompt_zs)
    results_zeroshot.append({
        "true_label": true_label_name,
        "predicted_label": pred_zs,
        "latency": lat_zs,
        "raw_response": raw_zs
    })

    # --- Few-Shot ---
    prompt_fs = create_fewshot_prompt(article_text, category_names, few_shot_prompt_examples)
    pred_fs, lat_fs, raw_fs = get_gemini_classification(prompt_fs)
    results_fewshot.append({
        "true_label": true_label_name,
        "predicted_label": pred_fs,
        "latency": lat_fs,
        "raw_response": raw_fs
    })

print("\nEvaluation complete.")


Starting evaluation...


  0%|          | 0/100 [00:00<?, ?it/s]

Error calling Gemini API: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 200\nPlease retry in 10.887319193s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 

In [ ]:
# --- 7. Calculate and Print Results ---
def calculate_metrics(results):
    correct = 0
    total_latency = 0
    valid_predictions = 0
    for res in results:
        total_latency += res['latency']
        if res['predicted_label'] is not None:
            valid_predictions += 1
            if res['true_label'] == res['predicted_label']:
                correct += 1
        # else: Treat parse errors as incorrect

    accuracy = (correct / len(results)) * 100 if len(results) > 0 else 0
    avg_latency = (total_latency / len(results)) * 1000 if len(results) > 0 else 0 # in ms
    parse_rate = (valid_predictions / len(results)) * 100 if len(results) > 0 else 0
    return accuracy, avg_latency, parse_rate

acc_zs, lat_zs_avg, parse_zs = calculate_metrics(results_zeroshot)
acc_fs, lat_fs_avg, parse_fs = calculate_metrics(results_fewshot)

print("\n--- Gemini Flash Performance on AG News ---")
print(f"Number of test samples: {len(test_sample)}")

print("\nZero-Shot Results:")
print(f"  Accuracy: {acc_zs:.2f}%")
print(f"  Avg. Latency: {lat_zs_avg:.2f} ms per request")
print(f"  Successfully Parsed Responses: {parse_zs:.2f}%")

print("\nFew-Shot Results (3 examples):")
print(f"  Accuracy: {acc_fs:.2f}%")
print(f"  Avg. Latency: {lat_fs_avg:.2f} ms per request")
print(f"  Successfully Parsed Responses: {parse_fs:.2f}%")


--- Gemini Flash Performance on AG News ---
Number of test samples: 100

Zero-Shot Results:
  Accuracy: 77.00%
  Avg. Latency: 430.89 ms per request
  Successfully Parsed Responses: 93.00%

Few-Shot Results (3 examples):
  Accuracy: 81.00%
  Avg. Latency: 436.01 ms per request
  Successfully Parsed Responses: 92.00%


In [ ]:
# --- 8. Save results to inspect failures
df_zs = pd.DataFrame(results_zeroshot)
df_fs = pd.DataFrame(results_fewshot)
df_zs.to_csv("gemini_flash_zeroshot_results.csv", index=False)
df_fs.to_csv("gemini_flash_fewshot_results.csv", index=False)
print("\nDetailed results saved to CSV files.")


Detailed results saved to CSV files.
